# Federal Parlamentary Transcriptions


##  Downloading Federal Parlamentary Transcriptions
It's a lot of data... Download it in a robust way
Please do not execute the command now...

In [ ]:
import sys
import time

import requests
import jsonlines

# open/write compressed files without hassle
from smart_open import open

# where to store the jsonline file locally
local_file_name = 'parlament_transcriptions.jsonl.bz2'

# highest transcription id to look for 
max_transcription_id = 260000


# download in chunks (default of odata server from parlament)
chunk = 1000

# Limit to "German" transcriptions (but never trust foreign data...)
url_template = 'https://ws.parlament.ch/odata.svc/Transcript/'
filter = "Language eq 'DE' and ID ge {lower}L and ID lt {upper}L"

with open(local_file_name,'w') as f:
    writer = jsonlines.Writer(f)
    for lower in range(0,max_transcription_id,chunk):
        downloaded = False
        while not downloaded:
            url = url_template.format(chunk=chunk,lower=lower, upper=lower+chunk)
            
            r = requests.get(url,
                             params={
                "$filter":filter.format(chunk=chunk,lower=lower, upper=lower+chunk),
                "$format":"json"})
            try:
                r.raise_for_status()
                # get the json data from the request's answer
                rjson = r.json()
                
                # some sanity checks...
                if not "d" in rjson or rjson["d"] == []:
                    print(f"No data found in {r.url}", file=sys.stderr)
                    downloaded = True
                    continue
                
                # inform the user about progress
                print(f"{len(rjson['d']['results'])} transcripts found in {r.url}", file=sys.stderr)
                
                # write a list of transcriptions in one go
                writer.write_all(rjson["d"]["results"])
                downloaded = True

            except requests.exceptions.HTTPError as e:
                print(f"{e}",file=sys.stderr)
                # let's be patient if server has temporary problems
                time.sleep(20)
            finally:
                # be nice to the server ...
                time.sleep(1)



In [ ]:
! bzcat parlament_transcriptions.jsonl.bz2 

or just download the already downloaded file

In [ ]:
! wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/parlament_transcriptions.jsonl.bz2

## Convert transcriptions into plain tab-separated text format 
Transcriptions are mixed XML content

In [ ]:
import requests
import jsonlines
import sys
import time
import re
from smart_open import open

# better XML processing than the built-in
from lxml import etree

local_file_name = 'parlament_transcriptions.jsonl.bz2'

local_tsv_file_name = 'parlament_transcriptions.tsv.bz2'

def flattened_text(text):
    """Return text with all whitespace characters normalized to space"""

    text_list = extract_text_list(text)
    return " ".join(re.sub(r'\s+',' ',t) for t in text_list).strip()

def extract_text_list(text):
    """
    Return textual contents of mixed content xmlrpclib

    see https://lxml.de/tutorial.html#using-xpath-to-find-text

    """
    xml = etree.fromstring(text)
    try:
        result = etree.XPath("//text()")(xml)
    except:
        print(f"XML PROBLEM: {xml}",file=sys.stderr)
        return "_"
    return result




with open(local_file_name,'r') as f:
    reader = jsonlines.Reader(f)
    with open(local_tsv_file_name, "w") as of:
        for i,obj in enumerate(reader):
            # normalize the XML transcripts into plain text
            normalized_text = flattened_text(obj["Text"]).strip() if obj["Text"] else "_"
            
            # let's use symbolic names for parlamentary group names for readability of the data
            parlamentary_group_name = re.sub(r'\W+','',obj.get("ParlGroupName") if obj["ParlGroupName"] else '_')
            speaker_full_name = re.sub(r'\W+','',obj["SpeakerFullName"]) if obj["SpeakerFullName"] else '_'
            
            # write tab-separated file
            print(parlamentary_group_name, speaker_full_name, normalized_text, sep="\t", file=of)

            # emit basic progress information
            if i % 10000 == 0:
                print(f"{i} transcripts processed", file=sys.stderr)

print('Done',file=sys.stderr)


## Turning XML content into simple plain text

In [ ]:
sample_xml = """<pd_text><p>[VS]</p>\n<p><b>Wahl der zweiten Vizepräsidentin des Nationalrates für 1999/2000</b></p>\n<p><b>Election de la deuxième vice-présidente du Conseil national pour 1999/2000</b></p>\n<p>[VS]</p>\n<p><b>Präsident</b> (Seiler Hanspeter, Präsident): Die sozialdemokratische Fraktion, unterstützt von der freisinnig-demokratischen, der SVP-, der christlichdemokratischen, der evangelischen und unabhängigen Fraktion, schlägt Ihnen Frau Maury Pasquier vor.</p>\n<p>[VS]</p>\n<p><i>Ergebnis der Wahl - Résultat du scrutin</i></p>\n<p>Ausgeteilte Wahlzettel - Bulletins délivrés .... 176</p>\n<p>eingelangt - rentrés .... 176</p>\n<p>leer - blancs .... 16</p>\n<p>ungültig - nuls .... 1</p>\n<p>gültig - valables .... 159</p>\n<p>absolutes Mehr - Majorité absolue .... 80</p>\n<p>[VS]</p>\n<p><i>Es wird gewählt - Est élue</i></p>\n<p>Maury Pasquier Liliane .... mit 130 Stimmen</p>\n<p>[VS]</p>\n<p><i>Ferner haben Stimmen erhalten - Ont en outre obtenu des voix</i></p>\n<p>Zapfl Rosmarie .... 12</p>\n<p>Verschiedene - Divers .... 17</p>\n<p>[VS]</p>\n<p><b>Präsident</b> (Seiler Hanspeter, Präsident): Ich gratuliere Frau Maury Pasquier zu ihrer guten Wahl. <i>(Beifall)</i></p>\n<p>[VS]</p>\n<p>[VS]</p>\n<p><b>Wahl der vier Stimmenzähler und der vier Ersatzstimmenzähler</b></p>\n<p><b>Election des quatre scrutateurs et des quatre scrutateurs suppléants</b></p>\n<p>[VS]</p>\n<p><b>Präsident</b> (Seiler Hanspeter, Präsident): Wir kommen zur Wahl der Stimmenzähler und der Ersatzstimmenzähler. Die Wahlvorschläge sind Ihnen ausgeteilt worden. Es findet eine Listenwahl statt. Die vier ersten Namen betreffen die Stimmenzähler, die vier weiteren Namen die Ersatzstimmenzähler.</p>\n<p>[VS]</p>\n<p><i>Ergebnis der Wahl - Résultat du scrutin</i></p>\n<p>Ausgeteilte Wahlzettel - Bulletins délivrés .... 156</p>\n<p>eingelangt - rentrés .... 154</p>\n<p>leer - blancs .... 0</p>\n<p>ungültig - nuls .... 0</p>\n<p>gültig - valables .... 154</p>\n<p>absolutes Mehr - Majorité absolue .... 78</p>\n<p>[PAGE 2376] [VS]</p>\n<p>Es werden gewählt - Sont élus</p>\n<p>[VS]</p>\n<p><i>Stimmenzähler - Scrutateurs</i></p>\n<p>Binder Max .... mit 140 Stimmen</p>\n<p>Günter Paul .... mit 131 Stimmen</p>\n<p>Lauper Hubert .... mit 150 Stimmen</p>\n<p>Tschuppert Karl .... mit 141 Stimmen</p>\n<p>[VS]</p>\n<p><i>Ersatzstimmenzähler - Scrutateurs suppléants</i></p>\n<p>Galli Remo .... mit 147 Stimmen</p>\n<p>Schmied Walter .... mit 141 Stimmen</p>\n<p>Tillmanns Pierre .... mit 141 Stimmen</p>\n<p>Wittenwiler Milli .... mit 151 Stimmen</p>\n<p>[VS]</p>\n<p><b>Präsident </b>(Seiler Hanspeter, Präsident): Ich gratuliere den Gewählten zu ihrer Wahl. <i>(Beifall)</i></p>\n</pd_text>"""

In [ ]:
extract_text_list(sample_xml)

In [ ]:
flattened_text(sample_xml)

In [ ]:
! wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/parlament_transcriptions.tsv.bz2